In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
from Exp3021.Inference import Inferencer
import IPython.display as ipd

In [2]:
inferencer = Inferencer(
    hp_path= './Exp3021/Hyper_Parameters.yaml',
    checkpoint_path= './results/Exp3021/Checkpoint/S_110000.pt',
    batch_size= 8
    )

2023-11-27 17:18:18,862 (Inference:71) INFO: Checkpoint loaded at 110000 steps.


In [3]:
texts = [    
    '산리오 띠부씰이 들어있는 산리오빵과 판박이껌.',
    '경량화 기술은 어렵네요. 잘 들으셨나요?'
    ]

In [4]:
audios = inferencer.Inference_Epoch(
    texts= texts
    )

for text, audio in zip(texts, audios):
    print(text)
    ipd.display(ipd.Audio(audio, rate= inferencer.hp.Sound.Sample_Rate))
    print()

[Inference]:   0%|          | 0/1 [00:00<?, ?it/s]

산리오 띠부씰이 들어있는 산리오빵과 판박이껌.



경량화 기술은 어렵네요. 잘 들으셨나요?


In [5]:
import tensorly
import torch
tensorly.set_backend('pytorch')

def Tucker_Decomposition_Conv1d(layer, rank):
    """ Gets a conv layer,
        returns a nn.Sequential object with the Tucker decomposition.
    """
    (core, [last, first]), _ = tensorly.decomposition.partial_tucker(
        layer.weight.data,
        modes=[0, 1],
        n_iter_max=2000000,
        rank=rank,
        init='svd'
        )

    # A pointwise convolution that reduces the channels from S to R3
    first_layer = torch.nn.Conv1d(
        in_channels=first.shape[0],
        out_channels=first.shape[1],
        kernel_size=1,
        stride=1,
        padding=0,
        dilation=layer.dilation,
        bias=False
        )

    # A regular 2D convolution layer with R3 input channels
    # and R3 output channels
    core_layer = torch.nn.Conv1d(
        in_channels=core.shape[1],
        out_channels=core.shape[0],
        kernel_size=layer.kernel_size,
        stride=layer.stride,
        padding=layer.padding,
        dilation=layer.dilation,
        bias=False
        )

    # A pointwise convolution that increases the channels from R4 to T
    last_layer = torch.nn.Conv1d(
        in_channels=last.shape[1],
        out_channels=last.shape[0],
        kernel_size=1,
        stride=1,
        padding=0,
        dilation=layer.dilation,
        bias=True
        )

    if not layer.bias is None:
        last_layer.bias.data = layer.bias.data

    first_layer.weight.data = first.permute(1, 0)[:, :, None]
    last_layer.weight.data = last[:, :, None]
    core_layer.weight.data = core

    new_layers = [first_layer, core_layer, last_layer]
    
    return torch.nn.Sequential(*new_layers)


In [6]:
print(inferencer.model.text_encoder.projection)
inferencer.model.text_encoder.projection = Tucker_Decomposition_Conv1d(inferencer.model.text_encoder.projection, rank= 128)
print(inferencer.model.text_encoder.projection)

Conv1d(192, 384, kernel_size=(1,), stride=(1,))
Sequential(
  (0): Conv1d(192, 128, kernel_size=(1,), stride=(1,), bias=False)
  (1): Conv1d(128, 128, kernel_size=(1,), stride=(1,), bias=False)
  (2): Conv1d(128, 384, kernel_size=(1,), stride=(1,))
)


c:\Users\codej\miniconda3\lib\site-packages\tensorly\decomposition\_tucker.py:160: Warning: Given only one int for 'rank' instead of a list of 2 modes. Using this rank for all modes.
  warnings.warn(message, Warning)


In [7]:
audios = inferencer.Inference_Epoch(
    texts= texts
    )

for text, audio in zip(texts, audios):
    print(text)
    ipd.display(ipd.Audio(audio, rate= inferencer.hp.Sound.Sample_Rate))
    print()

[Inference]:   0%|          | 0/1 [00:00<?, ?it/s]

산리오 띠부씰이 들어있는 산리오빵과 판박이껌.



경량화 기술은 어렵네요. 잘 들으셨나요?
